# Strong Lensing and Dark Matter Mass Profiles

AST 3414 - Spring 2026

## Introduction

One of the most striking predictions of general relativity is that mass curves spacetime, bending the path of light. When a massive object — a galaxy or cluster dominated by **dark matter** — sits between us and a distant background source, the result can be spectacular: arcs, multiple images, or even complete **Einstein rings**.

### What You'll Learn

1. Review the key equations of strong gravitational lensing
2. Implement a **Singular Isothermal Sphere (SIS)** lens model in Python
3. Load a real **HST image** of a galaxy-scale Einstein ring
4. **Fit** the Einstein radius to the observed ring
5. Convert your measurement into a **projected dark matter mass**

---
**This is a Pair Programming Lab** - Please work together with all members looking at the same copy of the code on one computer. One member will be the **Driver** who controls the keyboard. The others will be the **Navigator(s)** who reviews code and think strategically, giving instructions to the Drive.

Every 20-30 minutes an announcement will be made to switch roles.

Run the cell below to install any missing packages and import everything you will need.

In [ ]:
import subprocess, sys
for pkg in ['astropy', 'astroquery', 'scipy', 'matplotlib']:
    subprocess.run([sys.executable, '-m', 'pip', 'install', pkg, '-q'])

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Circle
from scipy.optimize import curve_fit, minimize
from scipy.ndimage import gaussian_filter

from astropy.io import fits
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
from astropy.visualization import ZScaleInterval, ImageNormalize, LogStretch

print('Setup complete ✓')

---
## Part 1. Gravitational Lensing

Before writing any code, make sure both partners can answer the following questions. Jot answers in the markdown cell below the questions.

### The Lens Equation

The fundamental equation of gravitational lensing relates the **true source position** $\boldsymbol{\beta}$ (angle on the sky) to the **observed image position** $\boldsymbol{\theta}$ via the **reduced deflection angle** $\boldsymbol{\alpha}(\boldsymbol{\theta})$:

$$\boldsymbol{\beta} = \boldsymbol{\theta} - \boldsymbol{\alpha}(\boldsymbol{\theta})$$

All angles are measured in the lens plane.

### The Einstein Radius

For a circularly symmetric lens with source exactly behind the lens ($\beta = 0$), the image is a complete ring at the **Einstein radius**:

$$\theta_E = \sqrt{\frac{4GM_{E}}{c^2} \frac{D_{ls}}{D_l \, D_s}}$$

where $M_E$ is the **mass projected within** $\theta_E$, and $D_l$, $D_s$, $D_{ls}$ are angular diameter distances to the lens, source, and between them.

### The SIS Model

A **Singular Isothermal Sphere** (SIS) is the simplest physically motivated model for a dark-matter-dominated galaxy halo. Its 3D density profile is:

$$\rho(r) = \frac{\sigma_v^2}{2\pi G r^2}$$

where $\sigma_v$ is the line-of-sight velocity dispersion. Crucially, the **reduced deflection angle for a SIS is constant**:

$$\alpha_\mathrm{SIS}(\theta) = \theta_E \quad \text{(directed toward the lens center)}$$

This means the lens equation for a SIS becomes:

$$\beta = \theta - \theta_E \frac{\theta}{|\theta|}$$

### Question 1
What does $\theta_E$ being larger tell you qualitatively about the lens mass?
(Answer in the cell below.)

---
## Part 2. Implementing the SIS Lens Model

You will build a **ray-tracing** simulation. The idea: for each pixel in the **image plane** (what the telescope sees), compute where that ray came from in the **source plane**. Then, place a Gaussian source in the source plane and see which image-plane pixels map back to it — those are the lensed arcs.

In [ ]:
# Grid parameters
# All angles are in arcseconds
N     = 500          # grid size (N x N pixels)
fov   = 5.0          # full field of view in arcseconds

# Build 1-D coordinate arrays from -fov/2 to +fov/2
x_1d = np.linspace(-fov/2, fov/2, N)
y_1d = np.linspace(-fov/2, fov/2, N)

# Create 2-D grids  (shape: N x N)
# theta_x[i,j] = x-coordinate of pixel (i,j) in the image plane
theta_x, theta_y = np.meshgrid(x_1d, y_1d)

# Radial distance from the lens centre for each pixel
theta_r = np.sqrt(theta_x**2 + theta_y**2)

# Avoid divide-by-zero at the exact center
theta_r_safe = np.where(theta_r == 0, 1e-10, theta_r)

print(f'Grid: {N}×{N}, FOV = {fov}"')
print(f'Pixel scale: {fov/N*1000:.1f} mas/pixel')

### SIS Deflection and Ray Tracing

Complete the function below. For a SIS, the deflection angle vector is:

$$\boldsymbol{\alpha}(\boldsymbol{\theta}) = \theta_E \, \hat{\boldsymbol{\theta}} = \theta_E \frac{\boldsymbol{\theta}}{|\boldsymbol{\theta}|}$$

Use the lens equation to map each image-plane point to its source-plane position.

In [ ]:
def sis_ray_trace(theta_x, theta_y, theta_E):
    """
    Ray-trace through a Singular Isothermal Sphere lens.

    Parameters
    ----------
    theta_x, theta_y : 2-D arrays  — image-plane coordinates [arcsec]
    theta_E          : float        — Einstein radius [arcsec]

    Returns
    -------
    beta_x, beta_y : 2-D arrays   — source-plane coordinates [arcsec]
    """
    theta_r = np.sqrt(theta_x**2 + theta_y**2)
    theta_r_safe = np.where(theta_r == 0, 1e-10, theta_r)

    # YOUR CODE HERE ────────────────────────────────────────────────────
    # Hint: compute the unit vector components (hat_x, hat_y)
    # then apply the SIS deflection: alpha_vec = theta_E * hat_theta
    # then apply the lens equation: beta = theta - alpha

    hat_x = theta_x / theta_r_safe
    hat_y = theta_y / theta_r_safe

    alpha_x = # ← fill in
    alpha_y = # ← fill in

    beta_x  = # ← fill in
    beta_y  = # ← fill in
    # ───────────────────────────────────────────────────────────────────
    return beta_x, beta_y


def gaussian_source(beta_x, beta_y, src_x, src_y, src_sigma):
    """
    Evaluate a circular Gaussian source in the source plane.

    Parameters
    ----------
    beta_x, beta_y          : source-plane coordinate grids [arcsec]
    src_x, src_y            : source centre position [arcsec]
    src_sigma               : source half-width (Gaussian sigma) [arcsec]

    Returns
    -------
    I : 2-D array — surface brightness in image plane
    """
    r2 = (beta_x - src_x)**2 + (beta_y - src_y)**2
    I  = np.exp(-r2 / (2 * src_sigma**2))
    return I

print('Functions defined. Test them in the next cell.')

The code below will visualise the simulated Einstein ring.

In [ ]:
theta_E  = 1.0
src_x    = 0.0
src_y    = 0.0
src_sigma = 0.05

beta_x, beta_y = sis_ray_trace(theta_x, theta_y, theta_E)
image = gaussian_source(beta_x, beta_y, src_x, src_y, src_sigma)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ext = [-fov/2, fov/2, -fov/2, fov/2]

ax = axes[0]
ax.imshow(gaussian_source(theta_x, theta_y, src_x, src_y, src_sigma),
          origin='lower', extent=ext, cmap='inferno')
ax.set_title('Source Plane (unlensed)')
ax.set_xlabel('arcsec'); ax.set_ylabel('arcsec')

ax = axes[1]
ax.imshow(image, origin='lower', extent=ext, cmap='inferno')
circle = Circle((0, 0), theta_E, fill=False, color='cyan', lw=1.5,
                linestyle='--', label=f'theta_E = {theta_E}"')
ax.add_patch(circle)
ax.legend(loc='upper right')
ax.set_title('Image Plane (lensed by SIS)')
ax.set_xlabel('arcsec'); ax.set_ylabel('arcsec')

plt.tight_layout()
plt.show()

### Question 2

Set `src_x = src_y = 0.0`. You will see a perfect, complete **Einstein ring** formed at radius $\theta_E = 1''$. This is the $\beta = 0$ case: the source lies exactly on the optical axis, so the azimuthal symmetry is unbroken and every point on the ring at radius $\theta_E$ is a valid image. The ring traces exactly the cyan dashed circle.

Next, increase `src_x` gradually to 0.5 arcseconds. At what value does the ring break into distinct arcs or images?

Finally, double `theta_E` to 2.0". What happens to the ring size and the total flux? Why?

---
## Part 3. Loading Real HST Data

You will work with an archival *Hubble Space Telescope* image of **SDSS J1430+4105**, a well-studied galaxy-scale Einstein ring from the **SLACS survey** (Bolton et al. 2006). The lens is a massive elliptical at $z_l = 0.285$ and the source is a blue star-forming galaxy at $z_s = 0.575$.

I already created a cutout image using data from the STScI MAST public archive. The image is a "drizzled" F814W (I-band) ACS/WFC mosaic with a pixel scale of ~0.05"/pixel.

In [ ]:
import urllib.request, os
DATA_URL   = ('https://raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/slacsJ1430p4105_cutout_image.fits')
LOCAL_FILE = 'slacsJ1430p4105_cutout_image.fits'
if not os.path.exists(LOCAL_FILE):
    print('Downloading HST image ...')
    urllib.request.urlretrieve(DATA_URL, LOCAL_FILE)
    print('Downloaded:', LOCAL_FILE)
else:
    print('File already exists:', LOCAL_FILE)

# Load the FITS data
# This cutout is a single-extension FITS (SCI extension).
hdu = fits.open(LOCAL_FILE)
hdu.info()
data = hdu[1].data.astype(float)
hdr  = hdu[1].header

# Pixel scale from WCS header (CD1_1 in deg/px, negative for RA axis)
px_scale = abs(hdr.get('CD1_1', hdr.get('CDELT1', 0.05/3600))) * 3600  # arcsec/px
print(f'Image shape : {data.shape}')
print(f'Pixel scale : {px_scale:.4f} arcsec/px')
print(f'Flux range  : [{data.min():.3f}, {data.max():.3f}]')

Visualise and crop the data.


In [ ]:
# Crop to the central region (Einstein ring is in the centre)
# For HST data we crop a 200x200 pixel cutout from the image centre.
cy, cx   = np.array(data.shape) // 2
half     = 100
cutout   = data[cy-half:cy+half, cx-half:cx+half]
px_scale = abs(hdr['CD1_1']) * 3600   # arcsec per pixel

ny, nx = cutout.shape
fov_cut = nx * px_scale
ext_cut = [-fov_cut/2, fov_cut/2, -fov_cut/2, fov_cut/2]

print(f'Cutout: {ny}×{nx} px  |  FOV = {fov_cut:.2f}"  |  {px_scale*1000:.1f} mas/px')

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Linear scale
axes[0].imshow(cutout, origin='lower', extent=ext_cut, cmap='gray')
axes[0].set_title('HST Image — Linear stretch')
axes[0].set_xlabel('arcsec'); axes[0].set_ylabel('arcsec')

# Log scale (better reveals faint ring)
norm_log = ImageNormalize(cutout - cutout.min() + 1e-6, stretch=LogStretch(a=10))
axes[1].imshow(cutout, origin='lower', extent=ext_cut, cmap='inferno', norm=norm_log)
axes[1].set_title('HST Image — Log stretch')
axes[1].set_xlabel('arcsec')

plt.tight_layout()
plt.show()

### Question 3

Before proceeding to the fitting, identify the **lens galaxy** (bright central core) and the **lensed arcs/ring**. Estimate the Einstein radius visually by measuring the ring radius in pixels. Convert to arcseconds using `px_scale`. Record your estimate here: $\theta_E^{\rm visual} \approx$ ____"

## Part 4. Subtract the lens galaxy and fit the Einstein ring

We model the lens galaxy with a **Moffat profile** plus a constant background:

$$I(r) = \frac{I_0}{\left(1 + r^2/\alpha^2\right)^{\beta}} + b$$

where $\alpha$ is the scale radius, $\beta$ controls how steeply the wings fall off, and $b$ is the background level. The Moffat profile was originally developed to describe stellar PSFs in ground-based imaging, but it also fits the PSF-broadened cores of compact galaxies well because its **power-law wings** ($I \propto r^{-2\beta}$ at large $r$) are a much better match to real galaxy profiles than the Gaussian's exponential cutoff.

The **full-width at half-maximum** of a Moffat profile is:
$$\mathrm{FWHM} = 2\alpha\sqrt{2^{1/\beta}-1}$$

We fit using only the pixels **outside the Einstein ring zone** (r > 0.5"), so the ring itself does not bias the galaxy model.


In [ ]:
# Build a coordinate grid matching the cutout
x_c  = np.linspace(-fov_cut/2, fov_cut/2, nx)
y_c  = np.linspace(-fov_cut/2, fov_cut/2, ny)
gx_c, gy_c = np.meshgrid(x_c, y_c)
gr_c = np.sqrt(gx_c**2 + gy_c**2)

# Moffat profile + constant background (provided — no fill-in here)
def moffat_model(gx, gy, I0, alpha, beta, cx0, cy0, bg):
    """
    2-D circular Moffat profile + constant background.

    I(r) = I0 * (1 + r^2/alpha^2)^(-beta) + bg

    Parameters
    ----------
    gx, gy       : coordinate grids [arcsec]
    I0           : peak surface brightness [e-/s]
    alpha        : Moffat scale radius [arcsec]
    beta         : power-law index (controls wing steepness)
    cx0, cy0     : centroid [arcsec]
    bg           : constant background [e-/s]

    """
    r2 = (gx - cx0)**2 + (gy - cy0)**2
    return I0 * (1.0 + r2 / alpha**2)**(-beta) + bg


def galaxy_residuals(params, gx, gy, data, mask):
    """
    Sum-of-squared residuals between Moffat model and data,
    evaluated only on pixels outside the ring mask.

    params = [I0, alpha, beta, cx0, cy0, bg]
    mask   : boolean array — True on pixels to EXCLUDE

    Note: Do NOT mask the central pixels here. The Moffat needs
    the bright central pixels to constrain I0, alpha, and beta.
    """
    I0, alpha, beta, cx0, cy0, bg = params
    if I0 < 0 or alpha <= 0 or beta <= 0.5:
        return 1e20

    # YOUR CODE HERE ────────────────────────────────────────────────────
    # 1. Compute model = moffat_model(gx, gy, I0, alpha, beta, cx0, cy0, bg)
    # 2. Return np.sum((data[~mask] - model[~mask])**2)

    model = # <- fill in
    return  # <- fill in
    # ────────────────────────────────────────────────────────────────────


# Fit mask: exclude only the Einstein ring zone
# (We keep the central pixels so the Moffat peak is well constrained.)
ring_mask = (gr_c > 0.5) & (gr_c < 1.8)

# Background estimate from the outer sky annulus
bg0 = np.median(cutout[gr_c > 3.5])

# Use scipy.optimize.minimize (method='Nelder-Mead') to minimise galaxy_residuals.
result_gal = minimize(
    galaxy_residuals,
    [cutout.max(), 0.15, 2.0, 0.0, 0.0, bg0],
    args=(gx_c, gy_c, cutout, ring_mask),
    method='Nelder-Mead',
    options={'maxiter': 60000, 'xatol': 1e-7, 'fatol': 1e-7}
)
p_gal = result_gal.x

# Initial guess: [I0,           alpha, beta, cx0, cy0, bg ]
#                [cutout.max(), 0.15,  2.0,  0.0, 0.0, bg0]
I0_fit, alpha_fit, beta_fit, cx_fit, cy_fit, bg_fit = p_gal
fwhm_fit  = 2 * alpha_fit * np.sqrt(2**(1/beta_fit) - 1)
lens_model    = moffat_model(gx_c, gy_c, *p_gal)
ring_residual = cutout - lens_model

print('── Best-fit Moffat + background parameters ─────────────────────────')
print(f'  I0     = {I0_fit:.4f} e-/s  (peak brightness)')
print(f'  alpha  = {alpha_fit:.4f}"  (scale radius)')
print(f'  beta   = {beta_fit:.4f}  (power-law index)')
print(f'  FWHM   = {fwhm_fit*1000:.0f} mas  (compare: HST ACS/WFC PSF ~ 100 mas)')
print(f'  cx,cy  = {cx_fit:.4f}", {cy_fit:.4f}"  (centroid)')
print(f'  bg     = {bg_fit:.5f} e-/s  (sky background)')

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))
for ax, img, title in zip(axes,
    [cutout, lens_model, ring_residual],
    ['Data', 'Galaxy Model (Moffat + bg)', 'Residual (ring)']):
    norm = ImageNormalize(img, interval=ZScaleInterval())
    ax.imshow(img, origin='lower', extent=ext_cut, cmap='inferno', norm=norm)
    ax.set_title(title)
    ax.set_xlabel('arcsec')
axes[0].set_ylabel('arcsec')
plt.tight_layout()
plt.show()

Define the model and residual function

In [ ]:
def model_image(params, gx, gy):
    """
    Build a lensed model image for a SIS lens + Gaussian source.

    Parameters
    ----------
    params : [theta_E, src_x, src_y, src_sigma, amplitude]
    gx, gy : coordinate grids [arcsec]

    Returns
    -------
    model : 2-D array (same shape as gx)
    """
    theta_E, src_x, src_y, src_sigma, amp = params

    # 1. Call sis_ray_trace to get source-plane coordinates
    beta_x, beta_y = sis_ray_trace(gx, gy, theta_E)

    # 2. Evaluate unlensed source brightness at each source-plane position
    img = gaussian_source(beta_x, beta_y, src_x, src_y, src_sigma)

    # 3. Convolve with an approximate HST PSF (Gaussian)
    psf_sigma_px = 1.5
    img_psf = gaussian_filter(img, sigma=psf_sigma_px)

    # 4. Scale by amplitude
    return amp * img_psf

def chi_squared(params, data, gx, gy, noise_sigma=0.005):
    """
    Compute χ² between model and data.

    Parameters
    ----------
    params      : model parameters (see model_image)
    data        : observed ring image (lens-subtracted)
    gx, gy      : coordinate grids
    noise_sigma : per-pixel noise level
    """
    theta_E = params[0]
    if theta_E <= 0:         # unphysical parameter guard
        return 1e20

    mod = model_image(params, gx, gy)
    chi2 = np.sum((data - mod)**2 / noise_sigma**2)
    return chi2

print('Model functions defined.')

Optimise the fit using chi-squared

In [ ]:
from scipy.optimize import minimize

# Initial guess
# [theta_E, src_x, src_y, src_sigma, amplitude]
p0 = [1.0,   0.0,   0.0,   0.07,    ring_residual.max()]

# Clip the ring residual so negatives don't confuse the fitter
ring_clipped = np.clip(ring_residual, 0, None)

# Estimate noise from empty corners of the image
corner = ring_clipped[:20, :20]
noise_sigma = corner.std() + 1e-5
print(f'Estimated noise σ = {noise_sigma:.5f}')

# Run the optimiser
result = minimize(
    chi_squared,
    p0,
    args=(ring_clipped, gx_c, gy_c, noise_sigma),
    method='Nelder-Mead',
    options={'xatol': 1e-4, 'fatol': 1e-3, 'maxiter': 5000, 'disp': True}
)

p_best = result.x
print('\nBest-fit parameters ──────────────────────────────────────────')
labels = ['θ_E [arcsec]', 'src_x [arcsec]', 'src_y [arcsec]',
          'src_σ [arcsec]', 'amplitude']
for lab, val in zip(labels, p_best):
    print(f'  {lab:20s} = {val:.4f}')
print(f'\n  χ² = {result.fun:.1f}')
print(f'   reduced χ² = {result.fun/(len(ring_clipped)-len(p0)):.1f}')

Visualise the best-fit model and residuals

In [ ]:
theta_E_fit = p_best[0]
best_model  = model_image(p_best, gx_c, gy_c)
final_resid = ring_clipped - best_model

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

for ax, img, title in zip(
    axes,
    [ring_clipped, best_model, final_resid],
    ['Data (lens-subtracted)', 'Best-fit SIS Model', 'Residual']):
    norm = ImageNormalize(img, interval=ZScaleInterval())
    ax.imshow(img, origin='lower', extent=ext_cut, cmap='inferno', norm=norm)
    c = Circle((0, 0), theta_E_fit, fill=False, color='cyan', lw=1.5, linestyle='--')
    ax.add_patch(c)
    ax.set_title(title)
    ax.set_xlabel('arcsec')

axes[0].set_ylabel('arcsec')
axes[1].text(0.05, 0.93, f'theta_E = {theta_E_fit:.3f}"',
             transform=axes[1].transAxes, color='white', fontsize=12,
             bbox=dict(facecolor='black', alpha=0.5, pad=3))

plt.tight_layout()
plt.savefig('lensing_fit_result.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nBest-fit Einstein radius: θ_E = {theta_E_fit:.3f} arcsec')

---
## Part 5. From angles to dark matter mass

Your fitted $\theta_E$ is more than a geometric measurement — it **directly encodes the projected mass** of the dark matter halo.

Rearranging the Einstein radius formula:

$$M_E = \frac{c^2}{4G} \, \theta_E^2 \, \frac{D_l \, D_s}{D_{ls}}$$

This is the **total projected mass** within $\theta_E$. Subtract the stellar mass to get the dark matter fraction.

First, we can use the cosmology calculator in astropy to find the angular diameter distances for the known redshifts of the lens and source.

In [ ]:
z_lens   = 0.285
z_source = 0.575

D_l  = cosmo.angular_diameter_distance(z_lens).to(u.Mpc)
D_s  = cosmo.angular_diameter_distance(z_source).to(u.Mpc)
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source).to(u.Mpc)

print(f'D_l  = {D_l:.1f}')
print(f'D_s  = {D_s:.1f}')
print(f'D_ls = {D_ls:.1f}')

print(f'\nNote: D_l + D_ls = {(D_l + D_ls):.1f}  ≠  D_s = {D_s:.1f}')

### Question 4

Why is it that the angular diameter distance of the observer to the lens plus the angular diameter distance of the lens to the source does not equal the angular diameter distance of the observer to the source?

Use your fit of the radius of the Einstein ring from above and the calculated angular diameter distances to find the total enclosed mass of the lens, physical radius of the Einstein ring, and dark matter fraction. You will need to enter the code to define the mass of the Einstein ring.

In [ ]:
from astropy.constants import G, c

# Convert θ_E from arcseconds to radians
theta_E_rad = (theta_E_fit * u.arcsec).to(u.rad).value

# YOUR CODE HERE ────────────────────────────────────────────────────────
# Compute M_E using the formula above.
# Hint: M_E = (c**2 / (4*G)) * theta_E_rad**2 * (D_l * D_s / D_ls)
# Use astropy units throughout so the dimensions work out automatically.
# Convert the final answer to solar masses using .to(u.Msun)

M_E = # ← fill in

# ───────────────────────────────────────────────────────────────────────

print(f'θ_E  = {theta_E_fit:.3f} arcsec')
print(f'M_E  = {M_E:.3e}')
print(f'M_E  = {M_E.value:.2e} M_☉')

In [ ]:
# Physical size of the Einstein radius at the lens
R_E_kpc = (theta_E_rad * D_l).to(u.kpc)
print(f'Physical Einstein radius R_E = {R_E_kpc:.2f}')

M_stellar = 1.0e11 * u.Msun
M_DM = M_E - M_stellar
f_DM = (M_DM / M_E).decompose()    # dimensionless

print(f'Total projected mass : M_E  = {M_E:.3e}')
print(f'Stellar mass         : M_*  = {M_stellar:.3e}')
print(f'Dark matter mass     : M_DM = {M_DM:.3e}')
print(f'Dark matter fraction : f_DM = {f_DM:.2f}  ({float(f_DM)*100:.0f}%)')
print('\nLiterature value (Auger+ 2010):  f_DM ≈ 0.30–0.50 within θ_E')

### Question 5
The stellar mass of the lens galaxy can be estimated from photometry. For SDSS J1430+4105, published studies give a stellar mass of roughly $M_* \approx 1.09 \times 10^{13} \, M_\odot$ within the Einstein radius, and a mean Einstein mass of $M_E \approx 5.37 \times 10^{13} \, M_\odot$ (Eichner et al. 2012). How does your inferred $f_{\rm DM}$ compare to the literature value? What systematic errors could bias your $\theta_E$ measurement? List at least three.